In [2]:
import os
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


# Extraction de patchs (Explication necessaire aprés..)

In [ ]:
def ext_patches(X_i, Y_i, patch_size):
    X_patches = []
    Y_targs = []
    ecart = patch_size
    w, h = X_i.shape
    
    #La je ne sais oas si je dois faire un padding de l'image pour justement prendre en compte tous les pixels de l'image
    
    for i in range(ecart, h - ecart):
        for j in range (ecart, w - ecart):
            patch = X_i(i-ecart:i+ecart+1)
            label = Y_i[i,j]
            X_patches.append(patch.flatten())
            Y_targs.append(label)
    
    return np.array(X_patches), np.array(Y_targs)

#  Création du dataset depuis les paires (images bruitées - ground truth) 

In [ ]:
noisy_dir = "C:/Users/Asus/Desktop/M1-IMA/PIMA/data/noisy"
GT_dir = "C:/Users/Asus/Desktop/M1-IMA/PIMA/data/ground_truth"
types = ["gaussian_std25", "gaussian_std80", "rician_s20", "rician_s50", "gaussian20_rician50"]

#on cherche alors d'avoir un dataset qui inclus toutes les images (donc le svm n'apprendra pas seulement d'une seule image hein!)

X_tot =[]
Y_tot = []

for fname in os.listdir(noisy_dir):
    if any(t in fname for t in types):
        gt_file = fname.split("_slice")[0] + "_slice" + fname.split("_slice")[1].split("_")[0] + "_clean.npy"
        noisy_path = os.path.join(noisy_dir, fname)
        gt_path = os.path.join(gt_dir, gt_file)
        
        if os.path.exists(gt_path):
            X_img = np.load(noisy_path)
            Y_img = np.load(gt_path)

            X_patch, y_patch = extract_patches(X_img, Y_img)

            X_total.append(X_patch)
            y_total.append(y_patch)
            
X_total = np.vstack(X_total)
y_total = np.concatenate(y_total)

print("Nombre total de patchs :", X_total.shape[0])

#  Entrainement


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=0.2, random_state=42)

model = SVR(kernel='rbf')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

#que le MSE for now!!
mse = mean_squared_error(y_test, y_pred)
print("MSE sur le test set :", mse)


##  Evaluation du modèle par : PSNR, SSIM, mutual information

In [ ]:
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

